# Notebook for Generating Investigative Datasets for SAE's + Copy Suppression


Components:
- Generation <- get data + model and create token df (Neel Style)
- Calculation of non-SAE intervention (eg: Ablation)
- Calculation of SAE based interventions (eg: Reconstruction of Query)



# Set Up

In [ ]:
from transformer_lens import HookedTransformer, utils
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
import torch
from typing import List, Tuple
from transformer_lens.components import HookPoint
import pandas as pd

from jaxtyping import Float
from torch import Tensor
import warnings

from tqdm import tqdm
from functools import partial
import os 
os.environ["TOKENIZERS_PARALLELISM"] = "false"


warnings.filterwarnings("ignore")
import plotly.express as px
import sys 
sys.path.append("..")
from sae_training.config import LanguageModelSAERunnerConfig
# from sae_training.sparse_autoencoder import CPU_Unpickler
from sae_training.sparse_autoencoder import SparseAutoencoder

## Utils

Thanks Neel!

In [ ]:
def list_flatten(nested_list):
    return [x for y in nested_list for x in y]

def make_token_df(tokens, len_prefix=5, len_suffix=1, model=None):

    str_tokens = [model.to_str_tokens(t) for t in tokens]
    unique_token = [[f"{s}/{i}" for i, s in enumerate(str_tok)] for str_tok in str_tokens]
    
    context = []
    batch = []
    pos = []
    label = []
    for b in range(tokens.shape[0]):
        # context.append([])
        # batch.append([])
        # pos.append([])
        # label.append([])
        for p in range(tokens.shape[1]):
            prefix = "".join(str_tokens[b][max(0, p-len_prefix):p])
            if p==tokens.shape[1]-1:
                suffix = ""
            else:
                suffix = "".join(str_tokens[b][p+1:min(tokens.shape[1]-1, p+1+len_suffix)])
            current = str_tokens[b][p]
            context.append(f"{prefix}|{current}|{suffix}")
            batch.append(b)
            pos.append(p)
            label.append(f"{b}/{p}")
    # print(len(batch), len(pos), len(context), len(label))
    return pd.DataFrame(dict(
        str_tokens=list_flatten(str_tokens),
        unique_token=list_flatten(unique_token),
        context=context,
        batch=batch,
        pos=pos,
        label=label,
    ))


## Load Model

In [ ]:

model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    # refactor_factored_attn_matrices=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

## Load Data

In [ ]:
LENGTH_RANDOM_TOKS = 4
TOKEN_OF_INTEREST = " John"
N_REPEAT_TOKENS = 3

def generate_random_token_prompt(n_random_tokens = 10, n_repeat_tokens = 3, token_of_interest: str = " John"):
    
    random_tokens = torch.randint(0, model.tokenizer.vocab_size, (n_random_tokens,)).to(model.cfg.device)
    # append the token id for " John"
    if token_of_interest is not None:
        john_token = torch.tensor(model.to_single_token(token_of_interest)).unsqueeze(0).to(model.cfg.device)
        random_tokens = torch.cat([john_token, random_tokens], dim=0)
    
    # repeat the tokens 
    random_tokens = random_tokens.repeat(n_repeat_tokens)
    
    # generate an index for each group of tokens
    random_token_groups = torch.arange(0, n_repeat_tokens).unsqueeze(-1).repeat(1, LENGTH_RANDOM_TOKS+1).flatten()
    
    return random_tokens, random_token_groups


In [ ]:
def get_webtext(seed: int = 420, dataset="stas/openwebtext-10k") -> List[str]:
    """Get 10,000 sentences from the OpenWebText dataset"""

    # Let's see some WEBTEXT
    raw_dataset = load_dataset(dataset)
    train_dataset = raw_dataset["train"]
    dataset = [train_dataset[i]["text"] for i in range(len(train_dataset))]

    # Shuffle the dataset (I don't want the Hitler thing being first so use a seeded shuffle)
    np.random.seed(seed)
    np.random.shuffle(dataset)

    return dataset

data = get_webtext()

# Load Sparse AutoEncoder

In [ ]:
# import wandb
# run = wandb.init()
# artifact = run.use_artifact('jbloom/mats_sae_training_gpt2_small_hook_q/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096:v13', type='model')
# artifact_dir = artifact.download()

### Loading Ours in

In [ ]:
from sae_training.utils import LMSparseAutoencoderSessionloader
from sae_training.sparse_autoencoder import SparseAutoencoder
# path = "checkpoints/ikig1wjm/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_32768.pkl"

# path="../artifacts/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096:v15/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096.pkl"#
# path="../artifacts/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096:v16/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096.pkl"
# path="../artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_24576:v56/final_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_24576.pkl"

# sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)
# hacky solution to saved with cuda load on mps:


path = "artifacts/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096:v13/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096.pkl"

import io
import pickle

path = "artifacts/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_65536:v28/1076002816_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_65536.pkl" 
# path = "checkpoints/peu1onjp/132669440_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_8192.pkl"
# path = "checkpoints/g2zrx9ho/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_8192.pkl"

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

with open(path, 'rb') as file:
    state_dict = CPU_Unpickler(file).load()

cfg = state_dict["cfg"].__dict__
cfg["device"] = "mps"
del cfg["d_sae"]
del cfg["tokens_per_buffer"]
cfg = LanguageModelSAERunnerConfig(**cfg)
sparse_autoencoder = SparseAutoencoder(cfg)
sparse_autoencoder.load_state_dict(state_dict["state_dict"])
del state_dict
del cfg


### Test Jacob's SAE

In [ ]:
# from transformer_lens.utils import download_file_from_hf
# from dataclasses import dataclass
# point, layer = "resid_pre", 10
# dic = utils.download_file_from_hf("jacobcd52/gpt2-small-sparse-autoencoders", f"gpt2-small_6144_{point}_{layer}.pt", force_is_torch=True)
# # sparse_autoencoder.load_state_dict(dic)
# dic.keys()


# @dataclass
# class SparseAutoencoderConfig:
#     d_sae: int
#     d_in: int
#     l1_coefficient: float
#     dtype: str
#     seed: int
#     device: str
#     model_batch_size: int
#     hook_point: str = "blocks.10.hook_resid_pre"
#     hook_point_layer: int = 10
    
# cfg = {
#     "d_sae": 6144,
#     "d_in": 768,
#     "l1_coefficient": 0.001,
#     "dtype": torch.float32,
#     "seed": 0,
#     "device": "mps",
#     "model_batch_size": 1028,
# }

# sparse_autoencoder_cfg = SparseAutoencoderConfig(**cfg)
# sparse_autoencoder = SparseAutoencoder(sparse_autoencoder_cfg)


# point, layer = "resid_pre", 10
# dic = download_file_from_hf("jacobcd52/gpt2-small-sparse-autoencoders", f"gpt2-small_6144_{point}_{layer}.pt", force_is_torch=True)
# sparse_autoencoder.load_state_dict(dic)


## Define Hooks

In [ ]:
LAYER_IDX, HEAD_IDX = (10, 7)
W_U = model.W_U.clone()
HEAD_HOOK_RESULT_NAME = utils.get_act_name("result", LAYER_IDX)
HEAD_HOOK_QUERY_NAME = utils.get_act_name("q", LAYER_IDX)
HEAD_HOOK_RESID_NAME = utils.get_act_name("resid_pre", LAYER_IDX)
# BATCH_SIZE = 10

def hook_to_ablate_head(head_output: Float[Tensor, "batch seq_len head_idx d_head"], hook: HookPoint, head = (LAYER_IDX, HEAD_IDX)):
    assert head[0] == hook.layer()
    assert "result" in hook.name
    head_output[:, :, head[1], :] = 0
    return head_output

def hook_to_reconstruct_query(
    head_input: Float[Tensor, "batch seq_len head_idx d_head"], 
    hook: HookPoint, 
    head = (LAYER_IDX, HEAD_IDX),
    reconstructed_query: Float[Tensor, "batch seq_len d_model"] = None,):
    assert head[0] == hook.layer()
    head_input[:, :, head[1], :] = reconstructed_query[:, :]
    return head_input

## Visualization

In [ ]:

def plot_attn(patterns, token_df, title="", facet_col_labels = ["Original", "Reconstructed"]):
    '''
    # patterns_original = cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    # patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    patterns_original = cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    both_patterns = torch.stack([patterns_original, patterns_reconstructed])
    plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")
    
    '''
    fig = px.imshow(patterns, text_auto=".2f", title=title,
                    facet_col=0,
                    color_continuous_midpoint=0,
                    color_continuous_scale="RdBu",
                    )
    
    tickvals = 1+ np.arange(patterns.shape[2])
    ticktext = token_df["unique_token"].tolist()
    
    # add tokens as x-ticks and y-ticks, for each facet
    # Update x-ticks and y-ticks for each facet
    for i in range(len(facet_col_labels)):
        fig.update_xaxes(
            dict(tickmode='array', tickvals=tickvals, ticktext=ticktext),
            row=1, col=i+1
        )
        fig.update_yaxes(
            dict(tickmode='array', tickvals=tickvals, ticktext=ticktext),
            row=1, col=i+1
        )
    
    
    # add facet col labels:
    for i, label in enumerate(facet_col_labels):
        fig.layout.annotations[i].text = label
        fig.layout.annotations[i].font.size = 20
        
    fig.update_layout(
        width=1200,
        height=800,
    )
    fig.show()
    

# plot_attn(patterns_original.detach().cpu(), token_df, title="Original Attention Distribution")
# plot_attn(patterns_reconstructed.detach().cpu(), token_df, title="Reconstructed Attention Distribution")

# Main Dataset Generation Func

In [ ]:
# Get ATTN results

def get_max_attn_token(tokens, cache, LAYER_IDX, HEAD_IDX):
    tokens = tokens.to("cpu")
    pattern_name = utils.get_act_name("pattern", LAYER_IDX)
    pattern = cache[pattern_name][0,HEAD_IDX].detach().cpu()
    max_idx_pos = pattern.argmax(dim=-1)
    max_idx_token_id = torch.gather(tokens, dim=-1, index=max_idx_pos.unsqueeze(-1).T)
    max_idx_tok = model.to_string(max_idx_token_id.T)
    max_idx_tok_value = pattern.max(dim=1).values
    return max_idx_pos[1:], max_idx_tok[1:], max_idx_tok_value[1:]


def kl_divergence_attention(y_true, y_pred):

    # Compute log probabilities for KL divergence
    log_y_true = torch.log(y_true + 1e-10)
    log_y_pred = torch.log(y_pred + 1e-10)

    return y_true * (log_y_true - log_y_pred)

In [ ]:

def eval_prompt(prompt: List) -> pd.DataFrame:
    '''
    Takes a list of strings as input.
    '''
    tokens = model.to_tokens(prompt)
    # tokens = tokens[:, :MAX_PROMPT_LEN]
    token_df = make_token_df(tokens[:,1:], model=model)
    
    # tokens = t.stack(tokens).to(device)
    
    # Basic Forward Pass
    loss, original_cache = model.run_with_cache(tokens, return_type="loss", loss_per_token=True)
    token_df['loss'] = loss.flatten().tolist()
    
    ## Collect ATTN Results
    max_idx_pos, max_idx_tok, max_idx_tok_value = get_max_attn_token(tokens, original_cache, LAYER_IDX, HEAD_IDX)
    token_df['max_idx_pos'] = max_idx_pos.flatten().tolist()
    token_df['max_idx_tok'] = max_idx_tok
    token_df['max_idx_tok_value'] = max_idx_tok_value.flatten().tolist()
    
    
    # Full Head Ablation
    ablated_loss = model.run_with_hooks(tokens, return_type="loss", loss_per_token=True, fwd_hooks=[(HEAD_HOOK_RESULT_NAME, hook_to_ablate_head)])
    token_df['ablated_loss'] = ablated_loss.flatten().tolist()
    token_df["loss_diff"] = token_df["ablated_loss"] - token_df["loss"]
    
    # Reconstruction of Query with SAE
    if "resid_pre" in sparse_autoencoder.cfg.hook_point:
        original_act = original_cache[sparse_autoencoder.cfg.hook_point]
        # token_df["q_norm"] = torch.norm(original_act, dim=-1)[:,1:].flatten().tolist()
        sae_out, feature_acts, _, mse_loss, _ = sparse_autoencoder(original_act)
        # token_df["rec_q_norm"] = torch.norm(sae_out, dim=-1)[:,1:].flatten().tolist()

        # need to generate query
        def replacement_hook(resid_pre, hook, new_resid_pre=sae_out):
            return new_resid_pre
        
        with model.hooks(fwd_hooks=[(HEAD_HOOK_RESID_NAME, replacement_hook)]):
            _, resid_pre_cache = model.run_with_cache(tokens, return_type="loss", loss_per_token=True)
            sae_out = resid_pre_cache[HEAD_HOOK_QUERY_NAME][:,:,HEAD_IDX]
        
        original_act = original_cache[HEAD_HOOK_QUERY_NAME][:,:,HEAD_IDX]
        mse_loss = (sae_out.float() - original_act.float()).pow(2).sum(-1)
        total_variance = original_act.pow(2).sum(-1)
        explained_variance = mse_loss/total_variance
        
    else:
        original_act = original_cache[sparse_autoencoder.cfg.hook_point][:,:,HEAD_IDX]
        token_df["q_norm"] = torch.norm(original_act, dim=-1)[:,1:].flatten().tolist()
        sae_out, feature_acts, _, mse_loss, _ = sparse_autoencoder(original_cache[sparse_autoencoder.cfg.hook_point][:,:,HEAD_IDX])
        token_df["rec_q_norm"] = torch.norm(sae_out, dim=-1)[:,1:].flatten().tolist()
        # norm_ratio = torch.norm(original_act, dim=-1)/ torch.norm(sae_out, dim=-1)
        total_variance = original_act.pow(2).sum(-1)
        explained_variance = mse_loss/total_variance
        
    num_active_features = (feature_acts > 0).sum(dim=-1)
    
    hook_fn = partial(hook_to_reconstruct_query, reconstructed_query=sae_out)
    with model.hooks(fwd_hooks=[(HEAD_HOOK_QUERY_NAME, hook_fn)]):
        _, cache_reconstructed_query = model.run_with_cache(tokens, return_type="loss", loss_per_token=True)
        max_idx_pos, max_idx_tok, max_idx_tok_value = get_max_attn_token(tokens, cache_reconstructed_query, LAYER_IDX, HEAD_IDX)
        
        # Get the KL Divergence of the attention distributions
        patterns_original = original_cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
        patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
        kl_result = kl_divergence_attention(patterns_original, patterns_reconstructed)
        kl_result = kl_result.sum(dim=-1)[1:].numpy()
    
    token_df['rec_q_max_idx_pos'] = max_idx_pos.flatten().tolist()
    token_df['rec_q_max_idx_tok'] = max_idx_tok
    token_df['rec_q_max_idx_tok_value'] = max_idx_tok_value.flatten().tolist()
    token_df['kl_divergence'] = kl_result.flatten().tolist()

    # add results to dataframe

    # SAE Metrics
    token_df['mse_loss'] = mse_loss.flatten()[1:].tolist()
    token_df['explained_variance'] = explained_variance.flatten()[1:].tolist()
    token_df['num_active_features'] = num_active_features.flatten()[1:].tolist()
    # print(feature_acts.shape)
    # token_df["ids_active_features"] = (feature_acts[0,1:] > 0)
    
    return token_df, original_cache, cache_reconstructed_query


In [ ]:

# Test on individual prompt (random repeating tokens)
random_tokens, random_token_groups = generate_random_token_prompt(n_random_tokens=LENGTH_RANDOM_TOKS, n_repeat_tokens=3, token_of_interest=" Mary")
prompt = model.to_string(random_tokens)
print(prompt)
token_df, original_cache, cache_reconstructed_query = eval_prompt([prompt])
print(token_df.columns)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
token_df[filter_cols].style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm",
)

# patterns_original = original_cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
# patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_original = original_cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")

In [ ]:
if not "resid_pre" in sparse_autoencoder.cfg.hook_point:
    original_act = original_cache[sparse_autoencoder.cfg.hook_point][:,:,HEAD_IDX]
    sae_out, feature_acts, _, mse_loss, _ = sparse_autoencoder(original_cache[sparse_autoencoder.cfg.hook_point][:,:,HEAD_IDX])
else:
    original_act = original_cache[sparse_autoencoder.cfg.hook_point]
    sae_out, feature_acts, _, mse_loss, _ = sparse_autoencoder(original_act)

In [ ]:
feature_acts.shape

In [ ]:
import pandas as pd
import plotly.express as px

def plot_line_with_top_10_labels(tensor, title="", n = 10):
    """
    Plots a line chart of the given tensor with the top 10 values annotated.

    :param tensor: A PyTorch tensor to be plotted.
    """
    # Convert the tensor to a Pandas series for easier processing
    data = pd.Series(tensor.detach().cpu().numpy())

    # Create the line chart
    fig = px.line(data, labels={'value': 'Activation Value', 'index': 'Feature Id'},
                    title=title)

    # Identify the top 10 values
    top_10_indices = data.nlargest(n).index

    # Annotate the top 10 values
    for idx in top_10_indices:
        fig.add_annotation(x=idx, y=data[idx],
                           text=f"{idx}",
                           showarrow=False, arrowhead=1,
                           ax=0, ay=-40)  # Adjust ax, ay as needed
    # Remove the legend
    fig.update_layout(showlegend=False)

    # Show the plot
    fig.show()

# Example usage
POS_INTEREST = feature_acts.shape[1] - 1
plot_line_with_top_10_labels(feature_acts[0, POS_INTEREST], "", 30)

In [ ]:

def plot_attn_score_by_feature(feature_ids, cache, token_df, title="", vals = None):
    '''
    '''

    k = cache[f"blocks.{LAYER_IDX}.attn.hook_k"][0,:(1+POS_INTEREST),HEAD_IDX]
    # score_contributions = sparse_autoencoder.W_enc[:,inds].T @ k.T
    score_contributions = sparse_autoencoder.W_dec[feature_ids] @ k.T
    if vals is not None:
        score_contributions = score_contributions * vals.unsqueeze(1)
    fig = px.imshow(score_contributions.detach().cpu(), 
                    color_continuous_scale="RdBu",
                    color_continuous_midpoint=0,
                    labels = dict(y="Feature", x="Token"),
                    text_auto=".2f", title="")
    # add xticks and y ticks
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=1+np.arange(score_contributions.shape[1]),
            ticktext=token_df["str_tokens"].tolist(),
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=np.arange(score_contributions.shape[0]),
            ticktext=list(feature_ids.detach().cpu().numpy()),
        ),
    )
    fig.update_layout(
        width=800,
        height=800,
    )

    fig.show()

feature_name = "All features"
vals, inds = torch.topk(feature_acts[0,POS_INTEREST],30)

print(inds)
plot_attn_score_by_feature(inds, original_cache, token_df, title="")#, vals=vals)

In [ ]:

def plot_unembed_score_by_feature(feature_ids, token_df, title="", vals = None):

    token_ids = model.to_tokens(token_df.str_tokens.to_list(), prepend_bos=False).squeeze()
    # W_U_normed = W_U / W_U.norm(dim=-1).unsqueeze(-1)
    resid_stream_projection =  sparse_autoencoder.W_dec[feature_ids] @ model.W_Q[LAYER_IDX, HEAD_IDX].T @ W_U[:,token_ids]
    
    if vals is not None:
        resid_stream_projection = resid_stream_projection * vals.unsqueeze(1)
    fig = px.imshow(resid_stream_projection.detach().cpu(), 
                    color_continuous_scale="RdBu",
                    color_continuous_midpoint=0,
                    labels = dict(y="Feature", x="Token"),
                    text_auto=".2f", title="")
    # add xticks and y ticks
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=np.arange(resid_stream_projection.shape[1]),
            ticktext=token_df["str_tokens"].tolist(),
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=np.arange(resid_stream_projection.shape[0]),
            ticktext=list(feature_ids.detach().cpu().numpy()),
        ),
    )
    fig.update_layout(
        width=800,
        height=800,
    )

    fig.show()

feature_name = "All features"
vals, inds = torch.topk(feature_acts[0,POS_INTEREST],10)

print(inds)
plot_unembed_score_by_feature(inds, token_df, vals=vals)

In [ ]:
def plot_feature_unembed_bar(feature_id, sparse_autoencoder, feature_name = ""):
    
    norm_unembed = model.W_U / model.W_U.norm(dim=0)[None: None]
    # feature_unembed = sparse_autoencoder.W_dec[feature_id] @ norm_unembed
    feature_unembed = sparse_autoencoder.W_dec[feature_id] @ model.W_Q[10,7].T @  model.W_U
    # feature_unembed = sparse_autoencoder.W_dec[feature_id] @  model.W_U
    # torch.topk(unembed_4795,10)

    feature_unembed_df = pd.DataFrame(
        feature_unembed.detach().cpu().numpy(),
        columns = [feature_name],
        index = [model.tokenizer.decode(i) for i in list(range(50257))]
    )

    feature_unembed_df = feature_unembed_df.sort_values(feature_name, ascending=False).reset_index().rename(columns={'index': 'token'})
    fig = px.bar(feature_unembed_df.head(20).sort_values(feature_name, ascending=True),
                 color_continuous_midpoint=0,
                 color_continuous_scale="RdBu",
            y = 'token', x = feature_name, orientation='h', color = feature_name, hover_data=[feature_name])

    fig.update_layout(
        width=800,
        height=600,
    )

    # fig.write_image(f"figures/{str(feature_id)}_{feature_name}.png")
    fig.show()
    
plot_feature_unembed_bar(46076, sparse_autoencoder, feature_name = "")


In [ ]:

def plot_qk_via_feature(feature_id, sparse_autoencoder, feature_name = "", highlight_tokens = []):
    eff_embed = model.W_E + model.blocks[0].mlp(model.blocks[0].ln2(model.W_E[None] + model.blocks[0].attn.b_O))
    eff_emb_in_key_space =  eff_embed @ model.W_K[LAYER_IDX,HEAD_IDX] @ sparse_autoencoder.W_dec[feature_id]
    # feature_unembed = sparse_autoencoder.W_dec[feature_id] @ model.W_Q[LAYER_IDX,HEAD_IDX].T @  model.W_U
    feature_unembed = sparse_autoencoder.W_enc[:,feature_id] @ model.W_Q[LAYER_IDX,HEAD_IDX].T @  model.W_U
    
    df = pd.DataFrame(dict(
        eff_emb_in_key_space=eff_emb_in_key_space[0].detach().cpu().numpy(),
        feature_unembed = feature_unembed.detach().cpu().numpy(),
        token = [model.tokenizer.decode(i) for i in range(50257)],
    ))
    
    df["token_of_interest"] = df["token"].isin(highlight_tokens)
    
    # add a column to df with text for the largest 10 values (positive and negative) 
    # that we can use to label these points
    top_10_key = df.sort_values("eff_emb_in_key_space", ascending=False).head(6)
    top_10_proj = df.sort_values("feature_unembed", ascending=False).head(6)
    
    top_10_key["text"] = top_10_key.apply(lambda x: f"{x['token']}", axis=1)
    top_10_proj["text"] = top_10_proj.apply(lambda x: f"{x['token']}", axis=1)
    
    # Merging the top and bottom points for annotation
    points_to_annotate = pd.concat([top_10_key, top_10_proj])

    fig = px.scatter(
        df,
        x="eff_emb_in_key_space",
        y = "feature_unembed",
        color="token_of_interest",
        color_continuous_scale="RdBu",
        # color="score_contributions",
        # text="text",
        # opacity=0.3,
        hover_data=["token"],
        labels=dict(eff_emb_in_key_space="Token to Feature Virtual Weight", feature_unembed="Unembed to Feature Virtual Weight"),
        title=f"Feature {feature_id} {feature_name}",
        template="plotly",
        marginal_x="histogram",
        marginal_y="histogram",
    )
    

    for _, row in points_to_annotate.iterrows():
        fig.add_annotation(x=row['eff_emb_in_key_space'], y=row['feature_unembed'],
                           text=row['text'], showarrow=False, arrowhead=1,
                           ax=20, ay=-40)

    
    fig.update_layout(
        width=1200,
        height=1200,
    )
    fig.show()

plot_qk_via_feature(46037, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())
# plot_qk_via_feature(23056, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())
# plot_qk_via_feature(17088, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())


## Realistic Example

In [ ]:
# prompt = "When John and Mary went to the shops, John gave the bag to"
# answer = " Mary"
# prompt = "All's fair in love and"
# answer = " war"
prompt = " The cat is cute. The dog is"
# prompt = " Alice, with her keen intelligence and artistic talent, discussed philosophy with Bob, who shared her intellect and also possessed remarkable culinary skills, while"
answer = " cute"
model.reset_hooks()
utils.test_prompt(prompt, answer, model)

with model.hooks(fwd_hooks=[(HEAD_HOOK_RESULT_NAME, hook_to_ablate_head)]):
    utils.test_prompt(prompt, answer, model)

In [ ]:
token_df, original_cache, cache_reconstructed_query = eval_prompt([prompt + answer])
print(token_df.columns)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
token_df[filter_cols].style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm")


In [ ]:
patterns_original = original_cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_original = original_cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")

In [ ]:
original_act = original_cache[sparse_autoencoder.cfg.hook_point][:,:,HEAD_IDX]
sae_out, feature_acts, _, mse_loss, _ = sparse_autoencoder(original_cache[sparse_autoencoder.cfg.hook_point][:,:,HEAD_IDX])
POS_INTEREST = feature_acts.shape[1]-2
print(POS_INTEREST)
plot_line_with_top_10_labels(feature_acts[0, POS_INTEREST], "")

In [ ]:
feature_name = "All features"
vals, inds = torch.topk(feature_acts[0,POS_INTEREST],10)
print(inds)
plot_attn_score_by_feature(inds, original_cache, token_df, title="")

In [ ]:
plot_unembed_score_by_feature(inds, token_df, vals=vals)

In [ ]:
# plot_qk_via_feature(58215, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())
# plot_qk_via_feature(57972, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())
plot_qk_via_feature(23287, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())

# plot_qk_via_feature(1704, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())
# plot_qk_via_feature(14106, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())
# plot_qk_via_feature(4092, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())
# plot_qk_via_feature(14106, sparse_autoencoder, feature_name = "", highlight_tokens=token_df.str_tokens.tolist())

In [ ]:
from pathlib import Path
import json
from typing import  Dict
from sae_analysis.visualizer import data_fns, html_fns
from sae_analysis.visualizer.data_fns import get_feature_data, FeatureData

In [ ]:

vocab_dict = model.tokenizer.vocab
vocab_dict = {v: k.replace("Ġ", " ").replace("\n", "\\n") for k, v in vocab_dict.items()}

vocab_dict_filepath = Path(os.getcwd()) / "vocab_dict.json"
if not vocab_dict_filepath.exists():
    with open(vocab_dict_filepath, "w") as f:
        json.dump(vocab_dict, f)
        

In [ ]:
inds

In [ ]:
from importlib import reload
from sae_analysis.visualizer import data_fns
reload(data_fns)

dataset="stas/openwebtext-10k"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# data = get_webtext()
raw_dataset = load_dataset(dataset)
train_dataset = raw_dataset["train"]
tokenized_data = utils.tokenize_and_concatenate(train_dataset, model.tokenizer, max_length=128)
tokenized_data = tokenized_data.shuffle(42)
all_tokens = tokenized_data["tokens"]


# Currently, don't think much more time can be squeezed out of it. Maybe the best saving would be to
# make the entire sequence indexing parallelized, but that's possibly not worth it right now.

max_batch_size = 32
total_batch_size = 512 * 10
feature_idx = list(inds.flatten().cpu().numpy())
# max_batch_size = 512
# total_batch_size = 16384
# feature_idx = list(range(1000))

tokens = all_tokens[:total_batch_size]

feature_data: Dict[int, FeatureData] = data_fns.get_feature_data(
    encoder=sparse_autoencoder,
    # encoder_B=sparse_autoencoder,
    model=model,
    hook_point=sparse_autoencoder.cfg.hook_point,
    hook_point_layer=sparse_autoencoder.cfg.hook_point_layer - 1,
    hook_point_head_index=sparse_autoencoder.cfg.hook_point_head_index,
    tokens=tokens,
    feature_idx=feature_idx,
    max_batch_size=max_batch_size,
    left_hand_k = 3,
    buffer = (5, 5),
    n_groups = 10,
    first_group_size = 20,
    other_groups_size = 5,
    verbose = True,
)


for test_idx in list(inds.flatten().cpu().numpy()):
    html_str = feature_data[test_idx].get_all_html()
    with open(f"data_{test_idx:04}.html", "w") as f:
        f.write(html_str)

## Run on whole dataset

In [ ]:
torch.mps.empty_cache()

In [ ]:
str_token_list = []
loss_list = []
ablated_loss_list = []

NUM_PROMPTS = 10
MAX_PROMPT_LEN = 100
# BATCH_SIZE = 10
dataframe_list = []
with torch.no_grad():
    for i in tqdm(range(NUM_PROMPTS)):
        
        # Get Token Data
        prompt = data[i]
        # new_str = data[BATCH_SIZE * i: BATCH_SIZE * (i + 1)]
        

        token_df, _, _= eval_prompt(prompt)
        dataframe_list.append(token_df)
        
df = pd.concat(dataframe_list)

print(df.shape)
print(df.columns)
df.head()

In [ ]:
df.sort_values(by="loss_diff", ascending=True).head(10).style.background_gradient(cmap='viridis', subset=["loss_diff", "mse_loss"])

In [ ]:
tmp = df.query("max_idx_tok == rec_q_max_idx_tok").query("max_idx_tok != '<|endoftext|>'")
print(df.shape)
print(tmp.shape[0])
tmp = tmp.sort_values("num_active_features", ascending=True).head(50)
tmp#.style.background_gradient(cmap='viridis', subset=["loss_diff", "num_active_features", "mse_loss", "kl_divergence", "q_norm", "rec_q_norm"])

In [ ]:
px.scatter(tmp, x="num_active_features", y="loss_diff", hover_data=["max_idx_tok", "max_idx_tok_value"], marginal_x="histogram", marginal_y="histogram")

In [ ]:
px.scatter(tmp, x="max_idx_tok_value", y="rec_q_max_idx_tok_value", hover_data=["max_idx_tok", "max_idx_tok_value"], marginal_x="histogram", marginal_y="histogram")

# Plot Results

In [ ]:
print(df.shape)
px.histogram(df, x="loss_diff", nbins=100, log_y=False, title="Loss Difference (Ablated - Original)")

In [ ]:
px.histogram(df, x="num_active_features", nbins=100, title="Loss Difference (Ablated - Original)")

In [ ]:
px.scatter(df,
           marginal_x="histogram",
           marginal_y="histogram",
           x="num_active_features", y="mse_loss", 
           log_y=True,
           log_x=True,
           title="Query Reconstuction Loss (MSE) vs. Number of Active Features")

# Drill Down

In [ ]:
vals, inds = torch.topk(feature_acts[0,POS_INTEREST],10)
tok_of_interest = token_df["str_tokens"][POS_INTEREST+1]
print(tok_of_interest)
tok_id = model.tokenizer.encode(tok_of_interest)[0]
projection_love = sparse_autoencoder.W_dec[inds] @ model.W_Q[10,7].T @  model.W_U[:,tok_id]
# projection_love = sparse_autoencoder.W_enc[:,inds].T @ model.W_Q[10,7].T @  model.W_U[:,love_id]
inds = inds.tolist()

print(projection_love.shape)
df = pd.DataFrame(dict(
    projection_love=projection_love.detach().cpu().numpy(),
    score_contributions = score_contributions[:,POS_INTEREST].detach().cpu().numpy() - score_contributions[:,0].detach().cpu().numpy(),
    feature_id=inds,
    activation=vals.detach().cpu().numpy(),
    rank=range(len(inds)),
))

fig = px.scatter(
    df,
    x="projection_love",
    y = "score_contributions",
    # color="feature_id",
    # color_continuous_scale="RdBu",
    color_continuous_midpoint=0,
    color="activation",
    hover_data=["feature_id"],
    labels=dict(projection_love="Feature-Token Unembed Proj", y="Activation", score_contributions="Attention Score Contribution"),
    template="plotly",
)

# add a black border around all points
fig.update_traces(marker=dict(line=dict(width=1, color="Black")))
# make all points slightly larger
fig.update_traces(marker=dict(size=12))
# increase font size
fig.update_layout(font=dict(size=18))
fig.update_layout(
    width=800,
    height=800,
)
fig.show()


In [ ]:

def plot_qk_via_feature(feature_id, sparse_autoencoder, feature_name = "", highlight_tokens = []):
    eff_embed = model.W_E + model.blocks[0].mlp(model.blocks[0].ln2(model.W_E[None] + model.blocks[0].attn.b_O))
    eff_emb_in_key_space =  eff_embed @ model.W_K[LAYER_IDX,HEAD_IDX] @ sparse_autoencoder.W_dec[feature_id]
    feature_unembed = sparse_autoencoder.W_dec[feature_id] @ model.W_Q[LAYER_IDX,HEAD_IDX].T @  model.W_U
    # feature_unembed = sparse_autoencoder.W_enc[:,feature_id] @ model.W_Q[LAYER_IDX,HEAD_IDX].T @  model.W_U
    
    df = pd.DataFrame(dict(
        eff_emb_in_key_space=eff_emb_in_key_space[0].detach().cpu().numpy(),
        feature_unembed = feature_unembed.detach().cpu().numpy(),
        token = [model.tokenizer.decode(i) for i in range(50257)],
    ))
    
    df["token_of_interest"] = df["token"].isin(highlight_tokens)
    
    # add a column to df with text for the largest 10 values (positive and negative) 
    # that we can use to label these points
    top_10_key = df.sort_values("eff_emb_in_key_space", ascending=False).head(6)
    top_10_proj = df.sort_values("feature_unembed", ascending=False).head(6)
    
    top_10_key["text"] = top_10_key.apply(lambda x: f"{x['token']}", axis=1)
    top_10_proj["text"] = top_10_proj.apply(lambda x: f"{x['token']}", axis=1)
    
    # Merging the top and bottom points for annotation
    points_to_annotate = pd.concat([top_10_key, top_10_proj])

    fig = px.scatter(
        df,
        x="eff_emb_in_key_space",
        y = "feature_unembed",
        color="token_of_interest",
        color_continuous_scale="RdBu",
        # color="score_contributions",
        # text="text",
        # opacity=0.3,
        hover_data=["token"],
        labels=dict(eff_emb_in_key_space="Token to Feature Virtual Weight", feature_unembed="Unembed to Feature Virtual Weight"),
        title=f"Feature {feature_id} {feature_name}",
        template="plotly",
        marginal_x="histogram",
        marginal_y="histogram",
    )
    

    for _, row in points_to_annotate.iterrows():
        fig.add_annotation(x=row['eff_emb_in_key_space'], y=row['feature_unembed'],
                           text=row['text'], showarrow=False, arrowhead=1,
                           ax=20, ay=-40)

    
    fig.update_layout(
        width=1200,
        height=1200,
    )
    fig.show()
    
# plot_qk_via_feature(inds[0], sparse_autoencoder, feature_name = "What's not to suppress here?")
plot_qk_via_feature(3985, sparse_autoencoder, feature_name = "", highlight_tokens=model.to_str_tokens(tokens[0,1:]))
# plot_qk_via_feature(1102, sparse_autoencoder, feature_name = "", highlight_tokens=model.to_str_tokens(tokens[0,1:]))
# plot_qk_via_feature(1664, sparse_autoencoder, feature_name = "", highlight_tokens=model.to_str_tokens(tokens[0,1:]))
# plot_qk_via_feature(3017, sparse_autoencoder, feature_name = "", highlight_tokens=model.to_str_tokens(tokens[0,1:]))
# plot_qk_via_feature(2282, sparse_autoencoder, feature_name = "", highlight_tokens=model.to_str_tokens(tokens[0,1:]))

In [ ]:
plot_qk_via_feature(inds[1], sparse_autoencoder, feature_name = "")

In [ ]:
plot_qk_via_feature(2433, sparse_autoencoder, feature_name = "")

In [ ]:
plot_qk_via_feature(2688, sparse_autoencoder, feature_name = "")

In [ ]:
# get the key proj via the

eff_embed.shape


def plot_proj_onto_embed_via_key(feature_id, sparse_autoencoder, feature_name = ""):
    
    eff_embed = model.W_E + model.blocks[0].mlp(model.blocks[0].ln2(model.W_E[None] + model.blocks[0].attn.b_O))
    
    eff_emb_in_key_space =  eff_embed @ model.W_K[LAYER_IDX,HEAD_IDX] @ sparse_autoencoder.W_dec[feature_id]

    feature_unembed_df = pd.DataFrame(
        eff_emb_in_key_space.T.detach().cpu().numpy(),
        columns = [feature_name],
        index = [model.tokenizer.decode(i) for i in list(range(50257))]
    )

    feature_unembed_df = feature_unembed_df.sort_values(feature_name, ascending=False).reset_index().rename(columns={'index': 'token'})
    fig = px.bar(feature_unembed_df.head(20).sort_values(feature_name, ascending=True),
                 color_continuous_midpoint=0,
                 color_continuous_scale="RdBu",
            y = 'token', x = feature_name, orientation='h', color = feature_name, hover_data=[feature_name])

    fig.update_layout(
        width=500,
        height=600,
    )

    # fig.write_image(f"figures/{str(feature_id)}_{feature_name}.png")
    fig.show()
plot_proj_onto_embed_via_key(2688, sparse_autoencoder, feature_name = "Famous People you Love")
plot_feature_unembed_bar(2688, sparse_autoencoder, feature_name = "Famous People you Love")